### Selenium Scraping

##### Gmarket 베스트 셀러 데이터 스크래핑 
- 전체 200개중 상위 10개 출력
- pandas 데이터를 분석할수 있는 python 라이브러리
    - pip3 install pandas
    - 데이터 프레임 생성 : df = pd.DataFrame(columns=["A","B"])
    - 데이터 추가 : df.loc[len(df)] = [{"A":1,"B":2}, {"A":3,"B":4}]

In [2]:
from selenium import webdriver
import pandas as pd

In [3]:
def get_gmarket_bestsellers(number=10):   #default number 10
    
    def to_int(price):
        return int(price.replace(",","").replace("원",""))
    
    driver =  webdriver.Chrome()
    driver.get('http://corners.gmarket.co.kr/Bestsellers')

    bestlist_selector = driver.find_elements_by_css_selector("#gBestWrap .best-list")[1] # ul 로 찍어서... 왜? 아이디 값은 유니크한 값이 너무 많아서
    items = bestlist_selector.find_elements_by_css_selector("li")[:number]
    
    df = pd.DataFrame(columns=["상품명","원가","판매가","이미지","상품링크"])
    
    for item in items:

        idx = item.find_element_by_css_selector("p").text # 순번
        img_link = item.find_element_by_css_selector("img").get_attribute("src") # 이미지 URL
        item_link = item.find_element_by_css_selector(".itemname").get_attribute("href") # 아이템 URL
        content = item.find_element_by_css_selector(".itemname").text # 아이템 텍스트
        o_price = item.find_element_by_css_selector(".item_price .o-price").text # 오리지날 가격
        s_price = item.find_element_by_css_selector(".item_price .s-price").text.split(" ")[0] # 판매 가격
        o_price = o_price if o_price else s_price # 원가 데이터가 없는경우 판매가를 대입
        
        data = {"상품명": content,
                "원가": to_int(o_price), 
                "판매가": to_int(s_price),
                "이미지": img_link, 
                "상품링크": item_link, }
        
        # Pandas 데이터 입력
        df.loc[len(df)] = data
        
    driver.close()
    return df

In [4]:
%%time
result = get_gmarket_bestsellers(200)
result.tail()

CPU times: user 2.25 s, sys: 138 ms, total: 2.39 s
Wall time: 39.5 s


In [5]:
# Pandas로 할인율 계산
result["할인율"] = 100 - (result["판매가"] / result["원가"]) * 100
result["할인율"] = result["할인율"].astype(dtype="float64")
result["할인율"] = round(result["할인율"], 2)
result = result.sort_values("할인율", ascending=False).reset_index().rename(columns={"index": "rank"})
result.head()

,rank,상품명,원가,판매가,이미지,상품링크,할인율
0,124,국산/봄신상/아동/내복/내의/주니어/실내복/잠옷,23000,6900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.0
1,129,[릴리푸리]여아의류/봄신상/릴리푸리/원피스/레깅스/치렝스,33000,9900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.0
2,62,[젤리스푼]아동복/봄신상오픈/상하복/티셔츠/레깅스/팬츠/점퍼,43000,12900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.0
3,67,삠뽀요/봄신상 원피스9900원 베이비 우주복 돌복,33000,9900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.0
4,111,[디즈니]디즈니 아동 캐릭터 실내화 슬리퍼 학생 덧신 샌들,23000,6900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.0


In [6]:
# utf-8로 인코딩한 csv 파일로 저장
# 한글을 인코딩 하기 위해서 (전세계 언어 엔코딩)
result.to_csv("gmarket.csv", encoding="utf-8")

##### Naver 메일 데이터 스크래핑
- 네이버 메일에 로그인한후 데이터 가져오기
- 로그인 같은 경우에는 request로 스크래핑이 불가능함

In [14]:
from selenium import webdriver
import pickle
import time

In [27]:
# 패스워드 저장하기
pw = "@Lucas23kb"
file = open("pw.pickle", "wb")
pickle.dump(pw, file)

In [28]:
main_url = "http://naver.com"
mail_url = "http://mail.naver.com"

In [40]:
# 브라우져 오픈 및 메인 사이트 이동
driver = webdriver.Chrome()
driver.get(main_url)

In [41]:
driver.find_element_by_css_selector(".ico_global_login").click()

In [42]:
# 아이디 패스워드 입력
pw = pickle.load(open("pw.pickle", "rb")) # pw 변수에 본인의 패스워드 입력
driver.find_element_by_css_selector( "#id" ).send_keys( "lucaseo1991" )
driver.find_element_by_css_selector( "#pw" ).send_keys( pw )

In [43]:
# 로그인 버튼 클릭
driver.find_element_by_css_selector(".btn_global").click()

In [44]:
# 메일 페이지로 이동
driver.get(mail_url)

In [45]:
# 메일 갯수
total_mail = driver.find_element_by_css_selector("#headTotalNum").text
total_mail = int(total_mail)
total_mail

561

In [46]:
# 안읽은 메일 갯수
not_read_mail = driver.find_element_by_css_selector("#headUnreadNum").text
not_read_mail = int(not_read_mail)
not_read_mail

0

In [47]:
# 메일 제목, 보낸사람 이름 가져오기
mails = driver.find_elements_by_css_selector(".mailList.sender_context > li")
print("전체 메일수 : ", len(mails))
df = pd.DataFrame(columns=["제목","보낸사람","메일링크"])
for idx, mail in enumerate(mails):
    title = mail.find_element_by_css_selector(".mail_title").text
    name = mail.find_element_by_css_selector(".name").text
    link = mail.find_element_by_css_selector(".subject > a").get_attribute("href")
    df.loc[len(df)] = { "제목":title, "보낸사람":name, "메일링크":link }    
df.tail()

전체 메일수 :  30


,제목,보낸사람,메일링크
25,[총동창회 직원 채용 공고],인하대학교총동창회,https://mail.naver.com/read/popup/?nMailId=5754
26,2017년 12월 KT email 명세서입니다.(010-29**-*505),KT,https://mail.naver.com/read/popup/?nMailId=5722
27,"[삼성증권] 서원영 고객님, '17.11월 월말잔고 및 거래내역 입니다.",삼성증권,https://mail.naver.com/read/popup/?nMailId=5695
28,New MASCHINE Expansion: BASEMENT ERA - East Co...,Native Instruments,https://mail.naver.com/read/popup/?nMailId=5681
29,(광고) YEEZY BOOST 350 V2 온라인 래플에 응모하세요.,adidas Originals,https://mail.naver.com/read/popup/?nMailId=5666


In [49]:
# marked 메일
mails = driver.find_elements_by_css_selector(".mailList.sender_context > li.mark")
print("전체 메일수 : ", len(mails))
df = pd.DataFrame(columns=["제목","보낸사람","메일링크"])
for idx, mail in enumerate(mails):
    title = mail.find_element_by_css_selector(".mail_title").text
    name = mail.find_element_by_css_selector(".name").text
    link = mail.find_element_by_css_selector(".subject > a").get_attribute("href")
    df.loc[len(df)] = { "제목":title, "보낸사람":name, "메일링크":link }    
df.tail()

전체 메일수 :  0


,제목,보낸사람,메일링크


In [38]:
# 읽은 메일
mails = driver.find_elements_by_css_selector(".mailList.sender_context > li:not(.notRead)")
print("total mails: ", len(mails))
df = pd.DataFrame(columns=["제목","보낸사람","메일링크"])
for idx, mail in enumerate(mails):
    title = mail.find_element_by_css_selector(".mail_title").text
    name = mail.find_element_by_css_selector(".name").text
    link = mail.find_element_by_css_selector(".subject > a").get_attribute("href")
    df.loc[len(df)] = { "제목":title, "보낸사람":name, "메일링크":link }    
df.tail()

total mails:  30


,제목,보낸사람,메일링크
25,[총동창회 직원 채용 공고],인하대학교총동창회,https://mail.naver.com/read/popup/?nMailId=5754
26,2017년 12월 KT email 명세서입니다.(010-29**-*505),KT,https://mail.naver.com/read/popup/?nMailId=5722
27,"[삼성증권] 서원영 고객님, '17.11월 월말잔고 및 거래내역 입니다.",삼성증권,https://mail.naver.com/read/popup/?nMailId=5695
28,New MASCHINE Expansion: BASEMENT ERA - East Co...,Native Instruments,https://mail.naver.com/read/popup/?nMailId=5681
29,(광고) YEEZY BOOST 350 V2 온라인 래플에 응모하세요.,adidas Originals,https://mail.naver.com/read/popup/?nMailId=5666


In [39]:
# 로그아웃
logout_url = driver.find_element_by_css_selector(".gnb_btn_login").get_attribute("href")
driver.get(logout_url)

In [ ]:
# 브라우져 닫기
driver.close()